# Memory
* Save your conversation with the LLM.

## Intro
* Ability to store information about past interactions.
* **Most of memory-related functionality in LangChain is marked as beta**. This is for two reasons:
    1. Most functionality are not production ready.
    2. Most functionality work with Legacy chains, not the newer LCEL syntax.
* **The main exception to this is the ChatMessageHistory functionality**. This functionality is largely production ready and does integrate with LCEL.

## LangChain documentation on Memory
* See the LangChain documentation page on Memory [here](https://python.langchain.com/v0.1/docs/modules/memory/).
* See the LangChain documentation page on how to use ChatMessageHistory with LCEL [here](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).
* See the LangChain documentation page on the various ChatMessageHistory integrations [here](https://python.langchain.com/v0.1/docs/integrations/memory/).

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-simple-chain.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-conversation-buffer**.

## Track operations
From now on, we can track the operations **and the cost** of this project from LangSmith:
* [smith.langchain.com](https://smith.langchain.com)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [19]:
#pip install python-dotenv

In [20]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
google_api_key = os.environ["GOOGLE_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [21]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [22]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [23]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

## Buffer Memory
* ConversationBufferMemory keeps a list of chat messages in a buffer and passes those into the prompt template.
* A buffer refers to a **temporary storage area in memory** used to hold data. Here, the buffer specifically holds chat messages before they are processed or used in some way, such as being passed into a prompt template.
* In simple terms, a buffer is like a waiting room for data. It's a temporary holding spot where data can stay until it's ready to be used or processed. Imagine you're cooking and you chop up vegetables before you need to cook them. Instead of chopping each vegetable right before it goes in the pan, you chop them all at once and put them aside on a cutting board. That cutting board with the chopped vegetables is like a buffer — it holds everything ready for you until you're ready to use it. This way, when you need the vegetables, they are all prepared and ready to go, making the cooking process smoother and faster.
* As you can see in the following example, ConversationBufferMemory was used mostly in the initial versions of LangChain.

In [24]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [25]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! Hi there! How are you doing today?', additional_kwargs={}, response_metadata={})],
 'text': 'Hello! Hi there! How are you doing today?'}

In [26]:
conversation({"question": "My name is Trung and I have moved 33 times."})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! Hi there! How are you doing today?
Human: My name is Trung and I have moved 33 times.

> Finished chain.


{'question': 'My name is Trung and I have moved 33 times.',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! Hi there! How are you doing today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Trung and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Wow, 33 times, Trung! That's a lot of moving! Are you a military family, or do you just enjoy a change of scenery? I'm curious to hear more about your experiences. How are you finding your current location?", additional_kwargs={}, response_metadata={})],
 'text': "Wow, 33 times, Trung! That's a lot of moving! Are you a military family, or do you just enjoy a change of scenery? I'm curious to hear more about your experiences. How are you finding your current location?"}

In [27]:
conversation({"question": "If my average moving distance was 100 miles, how many miles took all my moves?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! Hi there! How are you doing today?
Human: My name is Trung and I have moved 33 times.
AI: Wow, 33 times, Trung! That's a lot of moving! Are you a military family, or do you just enjoy a change of scenery? I'm curious to hear more about your experiences. How are you finding your current location?
Human: If my average moving distance was 100 miles, how many miles took all my moves?

> Finished chain.


{'question': 'If my average moving distance was 100 miles, how many miles took all my moves?',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! Hi there! How are you doing today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Trung and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Wow, 33 times, Trung! That's a lot of moving! Are you a military family, or do you just enjoy a change of scenery? I'm curious to hear more about your experiences. How are you finding your current location?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Okay, Trung. If you moved 33 times and each move averaged 100 miles, the total distance you've moved would be:\n\n33 moves * 100 miles/move = 33

In [28]:
conversation({"question": "Do you remember my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! Hi there! How are you doing today?
Human: My name is Trung and I have moved 33 times.
AI: Wow, 33 times, Trung! That's a lot of moving! Are you a military family, or do you just enjoy a change of scenery? I'm curious to hear more about your experiences. How are you finding your current location?
Human: If my average moving distance was 100 miles, how many miles took all my moves?
AI: Okay, Trung. If you moved 33 times and each move averaged 100 miles, the total distance you've moved would be:

33 moves * 100 miles/move = 3300 miles

So, you've moved a total of 3300 miles! That's quite a journey!
Human: Do you remember my name?

> Finished chain.


{'question': 'Do you remember my name?',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! Hi there! How are you doing today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Trung and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Wow, 33 times, Trung! That's a lot of moving! Are you a military family, or do you just enjoy a change of scenery? I'm curious to hear more about your experiences. How are you finding your current location?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Okay, Trung. If you moved 33 times and each move averaged 100 miles, the total distance you've moved would be:\n\n33 moves * 100 miles/move = 3300 miles\n\nSo, you've moved a total of 3300 miles! Th

In [29]:
print(memory.buffer)

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! Hi there! How are you doing today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='My name is Trung and I have moved 33 times.', additional_kwargs={}, response_metadata={}), AIMessage(content="Wow, 33 times, Trung! That's a lot of moving! Are you a military family, or do you just enjoy a change of scenery? I'm curious to hear more about your experiences. How are you finding your current location?", additional_kwargs={}, response_metadata={}), HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}), AIMessage(content="Okay, Trung. If you moved 33 times and each move averaged 100 miles, the total distance you've moved would be:\n\n33 moves * 100 miles/move = 3300 miles\n\nSo, you've moved a total of 3300 miles! That's quite a journey!", additional_kwargs={}, response_metadata={}),

In [30]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! Hi there! How are you doing today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Trung and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Wow, 33 times, Trung! That's a lot of moving! Are you a military family, or do you just enjoy a change of scenery? I'm curious to hear more about your experiences. How are you finding your current location?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Okay, Trung. If you moved 33 times and each move averaged 100 miles, the total distance you've moved would be:\n\n33 moves * 100 miles/move = 3300 miles\n\nSo, you've moved a total of 3300 miles! That's quite a journey!", additional_kwargs

## Conversation Buffer Window Memory
Similar to the previous one, but **you can limit the number of conversational exchanges stored in memory**. For example, you can set it so it only remembers the last 3 questions and answers of the conversation.

In [31]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [32]:
window_memory = ConversationBufferWindowMemory(k=3)

In [33]:
conversation_window = ConversationChain(
    llm=llm, 
    memory = window_memory,
    verbose=True
)

In [34]:
conversation_window({"input": "Hi, my name is Trung"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Trung
AI:

> Finished chain.


{'input': 'Hi, my name is Trung',
 'history': '',
 'response': "Hi Trung! It's nice to meet you. I don't have a name myself, but you can call me AI. I'm a large language model, trained by Google AI. I'm based on the Transformer architecture and have been trained on a massive dataset of text and code. This training allows me to communicate and generate human-like text in response to a wide range of prompts and questions. I can do things like summarize factual topics, create stories, and translate languages. What can I do for you today?"}

In [35]:
conversation_window({"input": "My favorite color is blue"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Trung
AI: Hi Trung! It's nice to meet you. I don't have a name myself, but you can call me AI. I'm a large language model, trained by Google AI. I'm based on the Transformer architecture and have been trained on a massive dataset of text and code. This training allows me to communicate and generate human-like text in response to a wide range of prompts and questions. I can do things like summarize factual topics, create stories, and translate languages. What can I do for you today?
Human: My favorite color is blue
AI:

> Finished chain.


{'input': 'My favorite color is blue',
 'history': "Human: Hi, my name is Trung\nAI: Hi Trung! It's nice to meet you. I don't have a name myself, but you can call me AI. I'm a large language model, trained by Google AI. I'm based on the Transformer architecture and have been trained on a massive dataset of text and code. This training allows me to communicate and generate human-like text in response to a wide range of prompts and questions. I can do things like summarize factual topics, create stories, and translate languages. What can I do for you today?",
 'response': 'That\'s a great choice, Trung! Blue is a very popular color. It\'s often associated with calmness, peace, and serenity. Fun fact: in color psychology, blue is also linked to trust, loyalty, and wisdom. Did you know that blue is one of the three primary colors in the RGB color model, which is used to create colors on computer screens and other digital displays? It\'s also a significant color in art; think of the famous 

In [36]:
conversation_window({"input": "My favorite animals are dogs"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Trung
AI: Hi Trung! It's nice to meet you. I don't have a name myself, but you can call me AI. I'm a large language model, trained by Google AI. I'm based on the Transformer architecture and have been trained on a massive dataset of text and code. This training allows me to communicate and generate human-like text in response to a wide range of prompts and questions. I can do things like summarize factual topics, create stories, and translate languages. What can I do for you today?
Human: My favorite color is blue
AI: That's a great choice, Trung! Blue is a very popular color. It's often associated with calmness, peace, and serenity. Fun fact: in 

{'input': 'My favorite animals are dogs',
 'history': 'Human: Hi, my name is Trung\nAI: Hi Trung! It\'s nice to meet you. I don\'t have a name myself, but you can call me AI. I\'m a large language model, trained by Google AI. I\'m based on the Transformer architecture and have been trained on a massive dataset of text and code. This training allows me to communicate and generate human-like text in response to a wide range of prompts and questions. I can do things like summarize factual topics, create stories, and translate languages. What can I do for you today?\nHuman: My favorite color is blue\nAI: That\'s a great choice, Trung! Blue is a very popular color. It\'s often associated with calmness, peace, and serenity. Fun fact: in color psychology, blue is also linked to trust, loyalty, and wisdom. Did you know that blue is one of the three primary colors in the RGB color model, which is used to create colors on computer screens and other digital displays? It\'s also a significant colo

In [37]:
conversation_window({"input": "I like to drive a vespa scooter in the city"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Trung
AI: Hi Trung! It's nice to meet you. I don't have a name myself, but you can call me AI. I'm a large language model, trained by Google AI. I'm based on the Transformer architecture and have been trained on a massive dataset of text and code. This training allows me to communicate and generate human-like text in response to a wide range of prompts and questions. I can do things like summarize factual topics, create stories, and translate languages. What can I do for you today?
Human: My favorite color is blue
AI: That's a great choice, Trung! Blue is a very popular color. It's often associated with calmness, peace, and serenity. Fun fact: in 

{'input': 'I like to drive a vespa scooter in the city',
 'history': 'Human: Hi, my name is Trung\nAI: Hi Trung! It\'s nice to meet you. I don\'t have a name myself, but you can call me AI. I\'m a large language model, trained by Google AI. I\'m based on the Transformer architecture and have been trained on a massive dataset of text and code. This training allows me to communicate and generate human-like text in response to a wide range of prompts and questions. I can do things like summarize factual topics, create stories, and translate languages. What can I do for you today?\nHuman: My favorite color is blue\nAI: That\'s a great choice, Trung! Blue is a very popular color. It\'s often associated with calmness, peace, and serenity. Fun fact: in color psychology, blue is also linked to trust, loyalty, and wisdom. Did you know that blue is one of the three primary colors in the RGB color model, which is used to create colors on computer screens and other digital displays? It\'s also a s

In [38]:
conversation_window({"input": "My favorite city is San Francisco"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite color is blue
AI: That's a great choice, Trung! Blue is a very popular color. It's often associated with calmness, peace, and serenity. Fun fact: in color psychology, blue is also linked to trust, loyalty, and wisdom. Did you know that blue is one of the three primary colors in the RGB color model, which is used to create colors on computer screens and other digital displays? It's also a significant color in art; think of the famous "Starry Night" painting by Vincent van Gogh, with its swirling blues. Do you have a particular shade of blue that you prefer, like sky blue, navy blue, or maybe a deep ocean blue? I find the variety of blue shades quite f

{'input': 'My favorite city is San Francisco',
 'history': 'Human: My favorite color is blue\nAI: That\'s a great choice, Trung! Blue is a very popular color. It\'s often associated with calmness, peace, and serenity. Fun fact: in color psychology, blue is also linked to trust, loyalty, and wisdom. Did you know that blue is one of the three primary colors in the RGB color model, which is used to create colors on computer screens and other digital displays? It\'s also a significant color in art; think of the famous "Starry Night" painting by Vincent van Gogh, with its swirling blues. Do you have a particular shade of blue that you prefer, like sky blue, navy blue, or maybe a deep ocean blue? I find the variety of blue shades quite fascinating!\nHuman: My favorite animals are dogs\nAI: Dogs! Excellent choice, Trung! They\'re definitely one of the most beloved animals on the planet. It\'s easy to see why; they\'re loyal, intelligent, and offer unconditional love. As a language model, I\'v

In [39]:
conversation_window({"input": "My favorite season is summer"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite animals are dogs
AI: Dogs! Excellent choice, Trung! They're definitely one of the most beloved animals on the planet. It's easy to see why; they're loyal, intelligent, and offer unconditional love. As a language model, I've processed countless texts about dogs, and I'm always amazed by the diversity of breeds. From tiny Chihuahuas to giant Great Danes, there's a dog for almost every lifestyle.

Did you know that dogs are descended from wolves? Scientists believe that the domestication of dogs began tens of thousands of years ago! Their keen sense of smell is legendary; they can detect scents that are imperceptible to humans. This ability is why they'

{'input': 'My favorite season is summer',
 'history': 'Human: My favorite animals are dogs\nAI: Dogs! Excellent choice, Trung! They\'re definitely one of the most beloved animals on the planet. It\'s easy to see why; they\'re loyal, intelligent, and offer unconditional love. As a language model, I\'ve processed countless texts about dogs, and I\'m always amazed by the diversity of breeds. From tiny Chihuahuas to giant Great Danes, there\'s a dog for almost every lifestyle.\n\nDid you know that dogs are descended from wolves? Scientists believe that the domestication of dogs began tens of thousands of years ago! Their keen sense of smell is legendary; they can detect scents that are imperceptible to humans. This ability is why they\'re used in so many important roles, like search and rescue, detecting drugs, and even identifying certain diseases.\n\nDo you have a favorite breed of dog? Or perhaps a favorite thing about dogs in general? I\'m always eager to learn more about what makes th

In [40]:
conversation_window({"input": "What is my favorite color?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I like to drive a vespa scooter in the city
AI: Ah, a Vespa scooter! That sounds like a fantastic way to navigate the city, Trung. I can almost picture you zipping through the streets, feeling the wind in your hair (assuming you're wearing a helmet, of course! Safety first!). Vespas are iconic, aren't they? They have such a distinctive design, a blend of retro charm and practical functionality.

Did you know that the Vespa was originally designed in Italy after World War II by Piaggio as an affordable mode of transportation for the masses? Its name, "Vespa," means "wasp" in Italian, supposedly because the sound of the engine resembled the buzzing of a wasp. The 

{'input': 'What is my favorite color?',
 'history': 'Human: I like to drive a vespa scooter in the city\nAI: Ah, a Vespa scooter! That sounds like a fantastic way to navigate the city, Trung. I can almost picture you zipping through the streets, feeling the wind in your hair (assuming you\'re wearing a helmet, of course! Safety first!). Vespas are iconic, aren\'t they? They have such a distinctive design, a blend of retro charm and practical functionality.\n\nDid you know that the Vespa was originally designed in Italy after World War II by Piaggio as an affordable mode of transportation for the masses? Its name, "Vespa," means "wasp" in Italian, supposedly because the sound of the engine resembled the buzzing of a wasp. The design was innovative for its time, featuring a unibody construction and a comfortable riding position.\n\nI\'ve processed a lot of information about urban transportation, and it seems Vespas are particularly popular in cities with narrow streets and dense traffic.

In [41]:
conversation_window({"input": "My favorite city is San Francisco"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite city is San Francisco
AI: San Francisco! What a wonderful choice, Trung! It's a city with such a unique character and vibrant culture. I've processed countless articles, books, and travel guides about San Francisco, and I'm always fascinated by its rich history and diverse neighborhoods. From the iconic Golden Gate Bridge to the charming Victorian houses of Alamo Square (the "Painted Ladies"), it's a city full of landmarks.

Did you know that San Francisco was originally founded in 1776 by Spanish colonists and was initially named Yerba Buena? It wasn't until the California Gold Rush in 1849 that the city experienced explosive growth and became a maj

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15
Please retry in 35.755639265s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 35
}
].



> Finished chain.


{'input': 'My favorite city is San Francisco',
 'history': 'Human: My favorite city is San Francisco\nAI: San Francisco! What a wonderful choice, Trung! It\'s a city with such a unique character and vibrant culture. I\'ve processed countless articles, books, and travel guides about San Francisco, and I\'m always fascinated by its rich history and diverse neighborhoods. From the iconic Golden Gate Bridge to the charming Victorian houses of Alamo Square (the "Painted Ladies"), it\'s a city full of landmarks.\n\nDid you know that San Francisco was originally founded in 1776 by Spanish colonists and was initially named Yerba Buena? It wasn\'t until the California Gold Rush in 1849 that the city experienced explosive growth and became a major economic and cultural hub. The Gold Rush transformed it from a small settlement into a bustling metropolis practically overnight!\n\nSan Francisco is also known for its hilly terrain, which makes for some breathtaking views, but also some challenging c

## Conversation Token Buffer Memory
Similar to the previous one, but this time **you can limit the number of tokens stored in memory**.

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [42]:
#!pip install tiktoken

In [43]:
from langchain.memory import ConversationTokenBufferMemory

In [44]:
token_memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=50
)

C:\Users\PC\AppData\Local\Temp\ipykernel_18664\130807733.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  token_memory = ConversationTokenBufferMemory(


In [45]:
conversation_token = ConversationChain(
    llm=llm, 
    memory = token_memory,
    verbose=True
)

## Conversation Summary Memory
**Stores a summary** of the previous conversational exchanges.

In [46]:
from langchain.memory import ConversationSummaryBufferMemory

In [47]:
summary_memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100
)

C:\Users\PC\AppData\Local\Temp\ipykernel_18664\3597302779.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryBufferMemory(


In [48]:
conversation_summary = ConversationChain(
    llm=llm, 
    memory = summary_memory,
    verbose=True
)

In [49]:
conversation_summary({"input": """Kurt Cobain dropped out of high school, 
then worked there as a janitor Even though he was by all accounts a slob, 
Kurt Cobain worked as a janitor at Weatherwax High School, not long after 
dropping out of that very school. The dancing janitor in the 
"Smells Like Teen Spirit" music video was an inside joke for 
those who knew of Cobain's old job.
"""})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Kurt Cobain dropped out of high school, 
then worked there as a janitor Even though he was by all accounts a slob, 
Kurt Cobain worked as a janitor at Weatherwax High School, not long after 
dropping out of that very school. The dancing janitor in the 
"Smells Like Teen Spirit" music video was an inside joke for 
those who knew of Cobain's old job.

AI:


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15
Please retry in 25.453417343s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 25
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<lo


> Finished chain.


{'input': 'Kurt Cobain dropped out of high school, \nthen worked there as a janitor Even though he was by all accounts a slob, \nKurt Cobain worked as a janitor at Weatherwax High School, not long after \ndropping out of that very school. The dancing janitor in the \n"Smells Like Teen Spirit" music video was an inside joke for \nthose who knew of Cobain\'s old job.\n',
 'history': '',
 'response': 'That\'s a really interesting detail about Kurt Cobain! I didn\'t know he worked as a janitor at his old high school after dropping out. It adds another layer to the irony and complexity of his persona, especially considering the "Smells Like Teen Spirit" video. The dancing janitor being an inside joke is a great tidbit. I wonder if he ever talked about that experience in interviews. I might try searching for some of his quotes about working as a janitor. Thanks for sharing that!'}

In [50]:
conversation_summary({"input": """
There were at least five different drummers in the band 
before Dave Grohl. Cobain and Novoselic were always members 
of Nirvana—formerly known as Skid Row, Pen Cap Chew, Bliss, 
and Ted Ed Fred—but finding a permanent drummer proved to be 
even harder than coming up with a decent band name. In the 
beginning, there was trivia answer Aaron Burckhard, who pissed 
off Cobain by getting Kurt's car impounded after being arrested 
for fighting with a police officer. Then there was Melvins 
drummer Dale Crover, who pounded the skins for Cobain and 
Novoselic on their first demo tape before moving to San Francisco. 
Next came Dave Foster, who got arrested for assaulting the 
son of the mayor of Cosmopolis, Washington. Burckhard briefly 
returned before announcing he was too hungover to practice one day. 
Then a mutual friend introduced Cobain and Novoselic to Chad 
Channing, who hung around for two years before the group's 
co-founders decided he wasn't cutting it anymore. Mudhoney 
drummer Dan Peters played on the "Sliver" single.
"""})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human shared a fact about Kurt Cobain working as a janitor at his old high school after dropping out, and that the dancing janitor in the "Smells Like Teen Spirit" video was an inside joke related to that. The AI found this interesting and expressed a desire to learn more about Cobain's experience as a janitor.
Human: 
There were at least five different drummers in the band 
before Dave Grohl. Cobain and Novoselic were always members 
of Nirvana—formerly known as Skid Row, Pen Cap Chew, Bliss, 
and Ted Ed Fred—but finding a permanent drummer proved to be 
even harder than coming up with a decent band name. In the 
beginning, there was trivia answer Aaron Bu

{'input': '\nThere were at least five different drummers in the band \nbefore Dave Grohl. Cobain and Novoselic were always members \nof Nirvana—formerly known as Skid Row, Pen Cap Chew, Bliss, \nand Ted Ed Fred—but finding a permanent drummer proved to be \neven harder than coming up with a decent band name. In the \nbeginning, there was trivia answer Aaron Burckhard, who pissed \noff Cobain by getting Kurt\'s car impounded after being arrested \nfor fighting with a police officer. Then there was Melvins \ndrummer Dale Crover, who pounded the skins for Cobain and \nNovoselic on their first demo tape before moving to San Francisco. \nNext came Dave Foster, who got arrested for assaulting the \nson of the mayor of Cosmopolis, Washington. Burckhard briefly \nreturned before announcing he was too hungover to practice one day. \nThen a mutual friend introduced Cobain and Novoselic to Chad \nChanning, who hung around for two years before the group\'s \nco-founders decided he wasn\'t cutting 

In [51]:
conversation_summary({"input": """
Back in Washington, Crover performed with Cobain and Novoselic 
on a seven date tour with Sonic Youth in August 1990, before 
Dave Grohl's band Scream broke up and Melvins frontman Buzz 
Osbourne introduced Grohl to Cobain and Novoselic, ending the 
vicious cycle of rotating drummers.
"""})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human shared a fact about Kurt Cobain working as a janitor at his old high school after dropping out, and that the dancing janitor in the "Smells Like Teen Spirit" video was an inside joke related to that. The AI found this interesting and expressed a desire to learn more about Cobain's experience as a janitor. Then the human detailed the numerous drummers who preceded Dave Grohl in Nirvana, including Aaron Burckhard, Dale Crover, Dave Foster, Chad Channing, and Dan Peters. The AI found this revolving door of drummers fascinating and summarized each drummer's contribution (or lack thereof) to the band, highlighting the instability of Nirvana's early days an

{'input': "\nBack in Washington, Crover performed with Cobain and Novoselic \non a seven date tour with Sonic Youth in August 1990, before \nDave Grohl's band Scream broke up and Melvins frontman Buzz \nOsbourne introduced Grohl to Cobain and Novoselic, ending the \nvicious cycle of rotating drummers.\n",
 'history': 'System: The human shared a fact about Kurt Cobain working as a janitor at his old high school after dropping out, and that the dancing janitor in the "Smells Like Teen Spirit" video was an inside joke related to that. The AI found this interesting and expressed a desire to learn more about Cobain\'s experience as a janitor. Then the human detailed the numerous drummers who preceded Dave Grohl in Nirvana, including Aaron Burckhard, Dale Crover, Dave Foster, Chad Channing, and Dan Peters. The AI found this revolving door of drummers fascinating and summarized each drummer\'s contribution (or lack thereof) to the band, highlighting the instability of Nirvana\'s early days an

In [52]:
print(summary_memory.buffer)

System: The human shared a fact about Kurt Cobain working as a janitor at his old high school after dropping out, and that the dancing janitor in the "Smells Like Teen Spirit" video was an inside joke related to that. The AI found this interesting and expressed a desire to learn more about Cobain's experience as a janitor. Then the human detailed the numerous drummers who preceded Dave Grohl in Nirvana, including Aaron Burckhard, Dale Crover, Dave Foster, Chad Channing, and Dan Peters. The AI found this revolving door of drummers fascinating and summarized each drummer's contribution (or lack thereof) to the band, highlighting the instability of Nirvana's early days and expressing curiosity about what happened to these drummers later in their careers. The human then explained that Dale Crover performed with Cobain and Novoselic on a seven-date tour with Sonic Youth in August 1990, before Buzz Osbourne from the Melvins introduced Dave Grohl to Cobain and Novoselic, ending the cycle of r

In [53]:
summary_memory.load_memory_variables({})

{'history': 'System: The human shared a fact about Kurt Cobain working as a janitor at his old high school after dropping out, and that the dancing janitor in the "Smells Like Teen Spirit" video was an inside joke related to that. The AI found this interesting and expressed a desire to learn more about Cobain\'s experience as a janitor. Then the human detailed the numerous drummers who preceded Dave Grohl in Nirvana, including Aaron Burckhard, Dale Crover, Dave Foster, Chad Channing, and Dan Peters. The AI found this revolving door of drummers fascinating and summarized each drummer\'s contribution (or lack thereof) to the band, highlighting the instability of Nirvana\'s early days and expressing curiosity about what happened to these drummers later in their careers. The human then explained that Dale Crover performed with Cobain and Novoselic on a seven-date tour with Sonic Youth in August 1990, before Buzz Osbourne from the Melvins introduced Dave Grohl to Cobain and Novoselic, endin

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-conversation-buffer.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-conversation-buffer.py